In [1]:
import requests, json

In [2]:
# Use cached results from previous user searches (Skyscanner API on RapidAPI)

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "a25c140655mshcc3de02bc48e78fp1d8ae5jsn716b89dc608a"
    }

## Start with one-way tickets and find cheapest option

* Two origins: Madrid and Zurich 
* Three city destinations (consider all airports in the three cities): London, Paris, Rome
* date: single date, & one-way

**goal**: chose cheapest city to fly to from Madrid and Zurich, i.e. city that will result in min total cost

In [59]:
# from Spain (ES-sky) - MAD (MAD-sky) & Switzerland (CH-sky) - Zurich (ZURI-sky)
origin_list = ["MAD-sky", "ZURI-sky"]

# where we are doing search from 
origin_country = "ES"

# list of possible destinations 
destination_list = ["LOND-sky", "PARI-sky", "ROME-sky"]

currency = "EUR"
locale = "en-US"
date = "2021-02-01" #yyyy-mm-dd

#for origin in origin_list:
    #print(f'For origin {origin}:')
    #print()
    
    #for destination in destination_list:
        #myurl = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/" + originCountry + "/" + currency + "/" + locale + "/"  + destination + "/" + origin + "/"+ date
        #response = requests.request("GET", myurl, headers=headers)

        #print(f'For destination {destination}:')
        #print(response.text)
        #print()
        #print('--------')
        #print()
    
    #print('########')
    #print()

In [131]:
# match airport IDs to names of origin and destination
# add to options list with 

airports = {}
carriers = {}
options = []
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/"

for origin in origin_list:
    for destination in destination_list:
        myurl = url + origin_country + "/" + currency + "/" + locale + "/"  + origin + "/" + destination + "/"+ date
        response = requests.request("GET", myurl, headers=headers)
        temp = json.loads(response.text)
        
        if("Quotes" in temp):
            for places in temp["Places"]:
                airports[places["PlaceId"]] = places["Name"] 
            
            for carrier in temp["Carriers"]:
                carriers[carrier["CarrierId"]] = carrier["Name"]
                
            for quotes in temp["Quotes"]:
                print("-----------")
                ori = quotes["OutboundLeg"]["OriginId"]
                dest = quotes["OutboundLeg"]["DestinationId"]
                carrier = quotes["OutboundLeg"]["CarrierIds"]
                price = quotes["MinPrice"]

                print(f"Journey:  {airports[ori]}  --> {airports[dest]}")
                print(f"Carrier: {carriers[carrier[0]]}")
                print(f"Price: {price} EUR")
                
                options.append({"origin": airports[ori], "destination": airports[dest], "price": price, 
                               "carrier": carriers[carrier[0]]})

-----------
Journey:  Madrid  --> London Gatwick
Carrier: Air Europa
Price: 38 EUR
-----------
Journey:  Madrid  --> Paris Orly
Carrier: Iberia
Price: 36 EUR
-----------
Journey:  Madrid  --> Rome Ciampino
Carrier: Ryanair
Price: 22 EUR
-----------
Journey:  Zurich  --> London Heathrow
Carrier: Iberia
Price: 64 EUR
-----------
Journey:  Zurich  --> London Heathrow
Carrier: SWISS
Price: 101 EUR
-----------
Journey:  Zurich  --> Paris Orly
Carrier: Air Europa
Price: 74 EUR
-----------
Journey:  Zurich  --> Paris Charles de Gaulle
Carrier: SWISS
Price: 104 EUR
-----------
Journey:  Zurich  --> Rome Fiumicino
Carrier: Air Europa
Price: 65 EUR
-----------
Journey:  Zurich  --> Rome Fiumicino
Carrier: Alitalia
Price: 81 EUR


In [132]:
len(options)

9

In [133]:
options

[{'origin': 'Madrid',
  'destination': 'London Gatwick',
  'price': 38,
  'carrier': 'Air Europa'},
 {'origin': 'Madrid',
  'destination': 'Paris Orly',
  'price': 36,
  'carrier': 'Iberia'},
 {'origin': 'Madrid',
  'destination': 'Rome Ciampino',
  'price': 22,
  'carrier': 'Ryanair'},
 {'origin': 'Zurich',
  'destination': 'London Heathrow',
  'price': 64,
  'carrier': 'Iberia'},
 {'origin': 'Zurich',
  'destination': 'London Heathrow',
  'price': 101,
  'carrier': 'SWISS'},
 {'origin': 'Zurich',
  'destination': 'Paris Orly',
  'price': 74,
  'carrier': 'Air Europa'},
 {'origin': 'Zurich',
  'destination': 'Paris Charles de Gaulle',
  'price': 104,
  'carrier': 'SWISS'},
 {'origin': 'Zurich',
  'destination': 'Rome Fiumicino',
  'price': 65,
  'carrier': 'Air Europa'},
 {'origin': 'Zurich',
  'destination': 'Rome Fiumicino',
  'price': 81,
  'carrier': 'Alitalia'}]

In [32]:
# self join -> there might be a more streamlined way of doing this (w pandas? look up)
dd_options = [options, options]

In [76]:
# Find common destination (city-wise) and calculate total price
price = []
dest_1_list = []
dest_2_list = []

for option_1 in dd_options[0]:
    
    #to avoid repeats, just read the first of the two origins
    if option_1["origin"] == "Madrid":
        
        for option_2 in dd_options[1]:
            # parse destination to get city (ex London) so that compare common cities, not just common airports
            city_1, airport_1 = option_1["destination"].split(" ", 1)
            city_2, airport_2 = option_2["destination"].split(" ", 1)

            if (option_1["origin"] != option_2["origin"]) & (city_1 == city_2) :

                ori_1 = option_1["origin"]
                ori_2 = option_2["origin"]
                dest_1 = option_1["destination"] 
                dest_2 = option_2["destination"]

                sum_price = option_1["price"] + option_2["price"]
                price.append(sum_price)
                dest_1_list.append(dest_1)
                dest_2_list.append(dest_2)

                print(f"Common destination:")
                print(f"Journey: {ori_1} -> {dest_1}")
                print(f"Journey: {ori_2} -> {dest_2}")
                print(f"Total price: {sum_price}")
                print("-----------")


Common destination:
Journey: Madrid -> London Stansted
Journey: Zurich -> London Heathrow
Total price: 95
-----------
Common destination:
Journey: Madrid -> London Stansted
Journey: Zurich -> London Heathrow
Total price: 132
-----------
Common destination:
Journey: Madrid -> London Gatwick
Journey: Zurich -> London Heathrow
Total price: 95
-----------
Common destination:
Journey: Madrid -> London Gatwick
Journey: Zurich -> London Heathrow
Total price: 132
-----------
Common destination:
Journey: Madrid -> Paris Orly
Journey: Zurich -> Paris Orly
Total price: 110
-----------
Common destination:
Journey: Madrid -> Paris Orly
Journey: Zurich -> Paris Charles de Gaulle
Total price: 141
-----------
Common destination:
Journey: Madrid -> Rome Ciampino
Journey: Zurich -> Rome Fiumicino
Total price: 89
-----------
Common destination:
Journey: Madrid -> Rome Ciampino
Journey: Zurich -> Rome Fiumicino
Total price: 103
-----------


In [104]:
# find cheapest destination (city-wise)
city_price = {}
airport_1_dict = {}
airport_2_dict = {}

for i, (d1, d2) in enumerate(zip(dest_1_list, dest_2_list)):
    # from mad
    city_1, airport_1 = d1.split(" ", 1)
    
    # from zuri
    city_2, airport_2 = d2.split(" ", 1)
    
    # if two destination cities are the same, then add price to corresponding dict entry in city_price
    if city_1 == city_2:
        if city_1 not in city_price:
            city_price[city_1] = [price[i]]
        else:
            city_price[city_1].append(price[i])  
            
        if city_1 not in airport_1_dict:
            airport_1_dict[city_1] = [airport_1]
        else:
            airport_1_dict[city_1].append(airport_1) 

        if city_2 not in airport_2_dict:
            airport_2_dict[city_2] = [airport_2]
        else:
            airport_2_dict[city_2].append(airport_2) 

In [92]:
city_price

{'London': [95, 132, 95, 132], 'Paris': [110, 141], 'Rome': [89, 103]}

In [105]:
airport_1_dict

{'London': ['Stansted', 'Stansted', 'Gatwick', 'Gatwick'],
 'Paris': ['Orly', 'Orly'],
 'Rome': ['Ciampino', 'Ciampino']}

In [106]:
airport_2_dict

{'London': ['Heathrow', 'Heathrow', 'Heathrow', 'Heathrow'],
 'Paris': ['Orly', 'Charles de Gaulle'],
 'Rome': ['Fiumicino', 'Fiumicino']}

In [109]:
cheapest_city_price = {}

for city in city_price:
    i = city_price[city].index(min(city_price[city]))
    cheapest_city_price[city] = min(city_price[city])
    cheapest_a1 = airport_1_dict[city][i]
    cheapest_a2 = airport_2_dict[city][i]

In [95]:
cheapest_city_price

{'London': 95, 'Paris': 110, 'Rome': 89}

In [110]:
cheapest_a1 

'Ciampino'

In [111]:
cheapest_a2

'Fiumicino'

In [113]:
# Find cheapest destination out of the 3 city options 

cheapest_dest = (min(cheapest_city_price, key=cheapest_city_price.get))
cheapest_price = min(cheapest_city_price.values())

print(f"Cheapest destination:")
print('------')
print(f"Journey: Madrid -> {cheapest_dest}, Airport: {cheapest_a1}")
print(f"Journey: Zurich -> {cheapest_dest}, Airport: {cheapest_a2}")
print(f"Total price: {cheapest_price} EUR")
print('------')


Cheapest destination:
------
Journey: Madrid -> Rome, Airport: Ciampino
Journey: Zurich -> Rome, Airport: Fiumicino
Total price: 89 EUR
------


additionally: recover the breakdown of price, i.e. Madrid-Ciampino and Zurich-Fiumicino

## Db for names

In [136]:
import pandas as pd

In [137]:
names = pd.read_csv('SkyscannerAPIplaces.csv', sep=',')

In [138]:
names.head()

,ID,Name
0,ES-sky,spain
1,MAD-sky,madrid
2,BCN-sky,barcelona
3,ALC-sky,alicante
4,AGP-sky,malaga


## Load database with IATA and municipalities

Data downloaded to SQL lite database from [OpenFlights](https://openflights.org/data.html) as `airports table`. Filtered and cleaned data to new table `airports_clean`.

In [174]:
from sqlalchemy import create_engine

In [273]:
# load data

engine = create_engine('sqlite:///Data/city_iata.db')

In [274]:
df = pd.read_sql_table('airports_cities_clean', engine)

In [275]:
df['iata_sky_code'] = df['iata_code'] + '-sky'
df.head()

,airport_name,latitude_deg,longitude_deg,country,region,city,iata_code,iata_sky_code
0,Utirik Airport,11.222000,169.852005,MH,MH-UTI,Utirik Island,UTK,UTK-sky
1,Ocean Reef Club Airport,25.325399,-80.274803,US,US-FL,Key Largo,OCA,OCA-sky
2,Pilot Station Airport,61.934601,-162.899994,US,US-AK,Pilot Station,PQS,PQS-sky
3,Crested Butte Airpark,38.851918,-106.928341,US,US-CO,Crested Butte,CSE,CSE-sky
4,LBJ Ranch Airport,30.251801,-98.622498,US,US-TX,Johnson City,JCY,JCY-sky


In [276]:
df['city']

0           Utirik Island
1               Key Largo
2           Pilot Station
3           Crested Butte
4            Johnson City
              ...        
8493    Ganjingzi, Dalian
8494              Tonghua
8495     Hunnan, Shenyang
8496                Yanji
8497     Laobian, Yingkou
Name: city, Length: 8498, dtype: object

In [227]:
#df['city_min'] = df['city'].astype(str).str.lower().str.replace('-', ' ').str.replace('.', ' ').str.replace(',', ' ').str.strip()

In [ ]:
# remove extra white spaces 

#for i,el in enumerate(df['city_min']):
   # df['city_min'][i] = " ".join(el.split())

How will users enter city names?

Best is probably to use a dropdown menu that filters down as user types city names, with in parenthesis the country ID (ex. US) in case there are multiple cities with the same name.

In [277]:
# city names that will appear to user 

df['city_user'] = df['city'] + ' (' + df['country'] + ')'

In [278]:
df

,airport_name,latitude_deg,longitude_deg,country,region,city,iata_code,iata_sky_code,city_user
0,Utirik Airport,11.222000,169.852005,MH,MH-UTI,Utirik Island,UTK,UTK-sky,Utirik Island (MH)
1,Ocean Reef Club Airport,25.325399,-80.274803,US,US-FL,Key Largo,OCA,OCA-sky,Key Largo (US)
2,Pilot Station Airport,61.934601,-162.899994,US,US-AK,Pilot Station,PQS,PQS-sky,Pilot Station (US)
3,Crested Butte Airpark,38.851918,-106.928341,US,US-CO,Crested Butte,CSE,CSE-sky,Crested Butte (US)
4,LBJ Ranch Airport,30.251801,-98.622498,US,US-TX,Johnson City,JCY,JCY-sky,Johnson City (US)
...,...,...,...,...,...,...,...,...,...
8493,Dalian Zhoushuizi International Airport,38.965698,121.539001,CN,CN-21,"Ganjingzi, Dalian",DLC,DLC-sky,"Ganjingzi, Dalian (CN)"
8494,Tonghua Sanyuanpu Airport,42.253889,125.703333,CN,CN-22,Tonghua,TNH,TNH-sky,Tonghua (CN)
8495,Shenyang Taoxian International Airport,41.639801,123.483002,CN,CN-21,"Hunnan, Shenyang",SHE,SHE-sky,"Hunnan, Shenyang (CN)"
8496,Yanji Chaoyangchuan Airport,42.882801,129.451004,CN,CN-22,Yanji,YNJ,YNJ-sky,Yanji (CN)


In [281]:
df[df['city'] == 'Paris']

,airport_name,latitude_deg,longitude_deg,country,region,city,iata_code,iata_sky_code,city_user
3747,Henry County Airport,36.338200,-88.382896,US,US-TN,Paris,PHT,PHT-sky,Paris (US)
3788,Cox Field,33.636600,-95.450798,US,US-TX,Paris,PRX,PRX-sky,Paris (US)
4324,Paris-Le Bourget Airport,48.969398,2.441390,FR,FR-IDF,Paris,LBG,LBG-sky,Paris (FR)
4326,Charles de Gaulle International Airport,49.012798,2.550000,FR,FR-IDF,Paris,CDG,CDG-sky,Paris (FR)
4329,Paris-Orly Airport,48.723333,2.379444,FR,FR-IDF,Paris,ORY,ORY-sky,Paris (FR)


In [288]:
comp = dict(method='zip', archive_name='city_codes.csv')

df.to_csv('Data/city_codes.zip', compression=comp)

In [289]:
df.to_csv('Data/city_codes.csv')

In [268]:
list(df.loc[df['city_user'] == 'London (GB)', 'iata_sky_code'])

['LTN-sky', 'BQH-sky', 'LGW-sky', 'LCY-sky', 'LHR-sky', 'STN-sky', 'NHT-sky']

In [261]:
city_list = ['London (GB)', 'Boston (US)']
city_id_list = []

for city in city_list:
    city_id = list(df.loc[df['city_user'] == city, 'iata_sky_code'])
    city_id_list.append(city_id)

city_id_list = [city for sub_city in city_id_list for city in sub_city]
city_id_list

['LTN_sky',
 'BQH_sky',
 'LGW_sky',
 'LCY_sky',
 'LHR_sky',
 'STN_sky',
 'NHT_sky',
 'BOS_sky',
 'BNH_sky']

In [270]:
df[df['city_clean'] == 'madrid']

,country,region,city,iata_code,latitude,longitude,iata_sky_code,city_clean,city_user
4368,ES,ES-M,Madrid,ECV,40.370701,-3.78514,ECV-sky,madrid,Madrid (ES)
4379,ES,ES-M,Madrid,MAD,40.471926,-3.56264,MAD-sky,madrid,Madrid (ES)
4394,ES,ES-M,Madrid,TOJ,40.496700,-3.44587,TOJ-sky,madrid,Madrid (ES)


In [293]:
df.loc[df['iata_code'] == 'CDG', 'city'].item()

'Paris'

In [338]:
columns = ['origin_sky_id', 'origin_iata_id', 'dest_sky_id', 'dest_iata_id', 'price', 'carrier']
df_outbound = pd.DataFrame(columns = columns)

df_outbound

,origin_sky_id,origin_iata_id,dest_sky_id,dest_iata_id,price,carrier


In [339]:
df_outbound = df_outbound.append({'origin_sky_id': 'MAD-sky', 'origin_iata_id':'MAD', 'dest_sky_id':'CDG-sky', 'dest_iata_id':'CDG', 'price':62, 'carrier': 'Air Europa'}, ignore_index=True)
df_outbound                      

,origin_sky_id,origin_iata_id,dest_sky_id,dest_iata_id,price,carrier
0,MAD-sky,MAD,CDG-sky,CDG,62,Air Europa


## Use get_data.py script

In [425]:
import importlib
import get_data
importlib.reload(get_data)

<module 'get_data' from '/Users/alexandra/Documents/DS_projects/flight_finder/get_data.py'>

In [428]:
params, df_outbound, df_inbound = get_data.main()

Processing user-defined parameters...

Please provide the two origin cities (separated by a comma): London (GB),Boston (US),
Please provide possible destinations (separated by a comma): Madrid (ES),Rome(IT),Paris (FR)
Please provide the desired outbound date (yyyy-mm-dd): 2021-02-01
Please provide the desired inbound date (yyyy-mm-dd), or enter a space if only one-way trip:  


  0%|          | 0/9 [00:00<?, ?it/s]


Requesting flight data...


100%|██████████| 9/9 [00:26<00:00,  2.90s/it]


Number of possible flights to be analyzed: 17

Done!


In [429]:
df_outbound

,origin_sky_id,origin_iata_id,dest_sky_id,dest_iata_id,price,carrier,origin_city_name,dest_city_name
0,LTN-sky,LTN,MAD-sky,MAD,54,Wizz Air,London,Madrid
1,LTN-sky,LTN,CDG-sky,CDG,154,Wizz Air,London,Paris
2,LTN-sky,LTN,ORY-sky,ORY,90,Wizz Air,London,Paris
3,LGW-sky,LGW,MAD-sky,MAD,46,Air Europa,London,Madrid
4,LGW-sky,LGW,CDG-sky,CDG,215,Air Europa,London,Paris
5,LGW-sky,LGW,ORY-sky,ORY,45,Air Europa,London,Paris
6,LCY-sky,LCY,MAD-sky,MAD,179,KLM,London,Madrid
7,LCY-sky,LCY,LBG-sky,LBG,4443,Flexflight,London,Paris
8,LCY-sky,LCY,CDG-sky,CDG,151,KLM,London,Paris
9,LCY-sky,LCY,ORY-sky,ORY,225,KLM,London,Paris


## Get top N flights with lowest prices

Select min price between flights within common destination.

In [432]:
origin_cities = df_outbound['origin_city_name'].unique()

In [555]:
df_outbound_origin1 = df_outbound[ df_outbound['origin_city_name'] == origin_cities[0] ]
df_outbound_origin2 = df_outbound[ df_outbound['origin_city_name'] == origin_cities[1] ]

In [557]:
num_options_origin1 = df_outbound_origin1.shape[0]
num_options_origin1

16

In [558]:
num_options_origin2 = df_outbound_origin2.shape[0]
num_options_origin2

1

In [600]:
columns = ['common_dest_id', 'common_dest_name', 'total_price']
for i in range(len(origin_cities)):
    columns.append('origin_id_'+str(i+1))
    columns.append('origin_name_'+str(i+1))
    columns.append('carrier_'+str(i+1))
    columns.append('price_'+str(i+1))
    
df_common_dest = pd.DataFrame(columns = columns)


In [601]:
for row1 in df_outbound_origin1.iterrows():
    for row2 in df_outbound_origin2.iterrows():
        if row1[1][7] == row2[1][7]:
            price_1 = row1[1][4]
            price_2 = row2[1][4]
            
            df_common_dest = df_common_dest.append({'common_dest_id': row1[1][3],
                                                    'common_dest_name': row1[1][7],
                                                    'total_price': price_1+price_2,
                                                    'origin_id_1': row1[1][1],
                                                    'origin_name_1': row1[1][6],
                                                    'carrier_1': row1[1][5],
                                                    'price_1': price_1,
                                                    'origin_id_2': row2[1][1],
                                                    'origin_name_2': row2[1][6],
                                                    'carrier_2': row[1][5],
                                                    'price_2': price_2
                                                   }, ignore_index=True)


In [603]:
df_common_dest = df_common_dest.sort_values(['total_price']).reset_index(drop=True)

In [604]:
df_common_dest

,common_dest_id,common_dest_name,total_price,origin_id_1,origin_name_1,carrier_1,price_1,origin_id_2,origin_name_2,carrier_2,price_2
0,MAD,Madrid,311,LGW,London,Air Europa,46,BOS,Boston,Ryanair,265
1,MAD,Madrid,319,LTN,London,Wizz Air,54,BOS,Boston,Ryanair,265
2,MAD,Madrid,321,STN,London,Ryanair,56,BOS,Boston,Ryanair,265
3,MAD,Madrid,340,LHR,London,British Airways,75,BOS,Boston,Ryanair,265
4,MAD,Madrid,444,LCY,London,KLM,179,BOS,Boston,Ryanair,265


In [ ]:
sorted_common_dest = df_common_dest.to_json(orient="index")
sorted_common_dest

In [606]:
parsed = json.loads(sorted_common_dest)
parsed


{'0': {'common_dest_id': 'MAD',
  'common_dest_name': 'Madrid',
  'total_price': 311,
  'origin_id_1': 'LGW',
  'origin_name_1': 'London',
  'carrier_1': 'Air Europa',
  'price_1': 46,
  'origin_id_2': 'BOS',
  'origin_name_2': 'Boston',
  'carrier_2': 'Ryanair',
  'price_2': 265},
 '1': {'common_dest_id': 'MAD',
  'common_dest_name': 'Madrid',
  'total_price': 319,
  'origin_id_1': 'LTN',
  'origin_name_1': 'London',
  'carrier_1': 'Wizz Air',
  'price_1': 54,
  'origin_id_2': 'BOS',
  'origin_name_2': 'Boston',
  'carrier_2': 'Ryanair',
  'price_2': 265},
 '2': {'common_dest_id': 'MAD',
  'common_dest_name': 'Madrid',
  'total_price': 321,
  'origin_id_1': 'STN',
  'origin_name_1': 'London',
  'carrier_1': 'Ryanair',
  'price_1': 56,
  'origin_id_2': 'BOS',
  'origin_name_2': 'Boston',
  'carrier_2': 'Ryanair',
  'price_2': 265},
 '3': {'common_dest_id': 'MAD',
  'common_dest_name': 'Madrid',
  'total_price': 340,
  'origin_id_1': 'LHR',
  'origin_name_1': 'London',
  'carrier_1': 'B

In [607]:
with open('Data/sorted_common_dest.txt', 'w') as outfile:
    json.dump(parsed, outfile)

## run calculate_cheapest.py

In [630]:
import importlib
import calculate_cheapest
importlib.reload(calculate_cheapest)

<module 'calculate_cheapest' from '/Users/alexandra/Documents/DS_projects/flight_finder/calculate_cheapest.py'>

In [620]:
params, df_outbound, df_inbound = get_data.main()

Processing user-defined parameters...

Please provide the two origin cities (separated by a comma): London (GB),Boston (US)
Please provide possible destinations (separated by a comma): Madrid (ES),Rome(IT),Paris (FR)
Please provide the desired outbound date (yyyy-mm-dd): 2021-02-01
Please provide the desired inbound date (yyyy-mm-dd), or enter a space if only one-way trip:  
How many flights would you like to see? (ex enter 3 to see top 3 cheapest flights) 2


  0%|          | 0/9 [00:00<?, ?it/s]


Requesting flight data...

Processing flight data...


100%|██████████| 9/9 [00:18<00:00,  2.02s/it]


Number of possible flights to be analyzed: 18

Done!


In [621]:
df_common_dest = calculate_cheapest.get_common_dest(df_outbound, df_inbound)

Sorting by price...


In [622]:
df_common_dest

,common_dest_id,common_dest_name,total_price,origin_id_1,origin_name_1,carrier_1,price_1,origin_id_2,origin_name_2,carrier_2,price_2
0,MAD,Madrid,311,LGW,London,Air Europa,46,BOS,Boston,jetBlue,265
1,MAD,Madrid,319,LTN,London,Wizz Air,54,BOS,Boston,jetBlue,265
2,MAD,Madrid,321,STN,London,Ryanair,56,BOS,Boston,jetBlue,265
3,ORY,Paris,325,LGW,London,Air Europa,45,BOS,Boston,jetBlue,280
4,MAD,Madrid,340,LHR,London,British Airways,75,BOS,Boston,jetBlue,265
5,CDG,Paris,352,LHR,London,British Airways,72,BOS,Boston,jetBlue,280
6,ORY,Paris,369,LHR,London,Iberia,89,BOS,Boston,jetBlue,280
7,ORY,Paris,371,STN,London,Ryanair,91,BOS,Boston,jetBlue,280
8,ORY,Paris,373,LTN,London,Wizz Air,93,BOS,Boston,jetBlue,280
9,CDG,Paris,388,LTN,London,Wizz Air,108,BOS,Boston,jetBlue,280


In [631]:
calculate_cheapest.print_top_flights(params, df_common_dest)

1st cheapest flight:
First Outbound Journey: London(LGW) -> Madrid(MAD)
with Air Europa at 46 EUR
Second Outbound Journey: Boston(BOS) -> Madrid(MAD)
with jetBlue at 265 EUR
For a total price of 311
-----------
2st cheapest flight:
First Outbound Journey: London(LTN) -> Madrid(MAD)
with Wizz Air at 54 EUR
Second Outbound Journey: Boston(BOS) -> Madrid(MAD)
with jetBlue at 265 EUR
For a total price of 319
-----------


## Use live prices

* Live Flights Search queries all flight providers regarding recent flight data, availability, and prices (updated but slow to return info)
* Browse Flight Searches gets information from skyscanner cache, might be slightly outdated but more detailed and immediate (more efficient for wide range of dates and destinations). Returns cheapeast quote for each origin and destination per day. Popular routes updated frequently vs less popular routes and dates.


Possible option: Do Browse Flight Search, then once have chosen destination and date combination do Live Flight Search to find up to date availability and price (most accurate that Skyscanner provides).

In [412]:
headers = {
    "X-RapidAPI-Host" : "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    "X-RapidAPI-Key" : "a25c140655mshcc3de02bc48e78fp1d8ae5jsn716b89dc608a",
    "Content-Type" : "application/x-www-form-urlencoded"
    }

data = {'country' : 'ES',
        'currency' :'EUR',
        'locale' : 'en-US',
        'originPlace':'MAD-sky',
        'destinationPlace' : 'ZRH-sky',
        'outboundDate' : '2021-02-01',
        'adults': 1,
        'apikey': 'prtl6749387986743898559646983194'
       }

url_live = 'https://partners.api.skyscanner.net/apiservices/pricing/v1.0'

response_live = requests.post(url_live, data = data, headers= headers)
temp_live = json.loads(response_live.text)

temp_live

{}

In [419]:
response_live.status_code

201

In [416]:
response_live.text

'{ }'

In [418]:
url_browse = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/"


headers_browse = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "a25c140655mshcc3de02bc48e78fp1d8ae5jsn716b89dc608a"
    }

myurl = url_browse + origin_country + "/" + currency + "/" + locale + "/"  + 'MAD-sky/ZRH-sky/2021-02-01'
response = requests.request("GET", myurl, headers=headers_browse)
temp_browse = json.loads(response.text)

if("Quotes" in temp_browse):
    for places in temp_browse["Places"]:
        airports[places["PlaceId"]] = places["Name"] 

    for carrier in temp_browse["Carriers"]:
        carriers[carrier["CarrierId"]] = carrier["Name"]

    for quotes in temp_browse["Quotes"]:
        print("-----------")
        ori = quotes["OutboundLeg"]["OriginId"]
        dest = quotes["OutboundLeg"]["DestinationId"]
        carrier = quotes["OutboundLeg"]["CarrierIds"]
        price = quotes["MinPrice"]

        print(f"Journey:  {airports[ori]}  --> {airports[dest]}")
        print(f"Carrier: {carriers[carrier[0]]}")
        print(f"Price: {price} EUR")


-----------
Journey:  Madrid  --> Zurich
Carrier: Air Europa
Price: 35 EUR


**EDIT: LIVE PRICES HAS BEEN REMOVED AS OF APR 2020**

We're reaching out to inform you of a change in service for the Skyscanner Flight Search API on RapidAPI. Due to a change in strategy regarding Skyscanner's API, and to ensure that those using this service are doing so for the purpose helping travelers find and purchase flights, RapidAPI will no longer be offering the Live Search endpoint (generating life quotes of flight prices) as part of the Skyscanner API offering. Starting on April 1st, 2020 the Live Search endpoint will no longer be available to RapidAPI users. We strongly recommend you make the appropriate adjustments to transition all usage to the Browse endpoint in order to keep your application running properly. If you have a current travel booking application in production, are properly redirecting users to Skyscanner for booking, and need assistance in maintaining access to the Live endpoint;

please complete the following form to provide us with more information -
httpsJ/rapidapi.typeform.com/to/xcM3Gw
We'll route your information to the Skyscanner team for review. It is important to note that any decision made regarding allowing some users to maintain access to the Live Search endpoint is fully at Skyscanner's discretion and beyond RapidAPI's ability to accommodate. Users are also free to apply to Skyscanner's affiliate program to access additional resources that can enable travel functionality. Thank you for your understanding and please do reach out with any questions/concerns.

Thank you, The RapidAPI